# Предобработка данных

Подключим необходимые библиотеки.

In [54]:
import pandas as pd
from typing import List

Загрузим размеченный набор данных и посмотрим на него. При этом в предоставленных данных отсутствуют названия столбцов, поэтому назовем их сами.

In [68]:
data_markedup_path = '../data/payments_training.tsv'

data_markedup = pd.read_csv(data_markedup_path, encoding='utf-8', header=None,  names=['index', 'date', 'cash', 'description', 'type'], sep='\t', index_col=0)

In [69]:
data_markedup

,date,cash,description,type
index,,,,
1,07.11.2024,15300.00,За участие в конференции в г. Майкоп по догово...,SERVICE
2,07.11.2024,"40200,00",За оказание услуг по договору №79-02726В от 01...,SERVICE
3,07.11.2024,1440-00,Оплата за Порошок стиральный Ariel Color autom...,NON_FOOD_GOODS
4,07.11.2024,240000000-00,Возврат денежных средств по договору займа №04...,LOAN
5,07.11.2024,1360000.00,"Оплата Дог №452 от 13/03/2021, согл. Сч 0745-2...",NOT_CLASSIFIED
...,...,...,...,...
496,07.11.2024,2610.00,Оплата налогов,TAX
497,07.11.2024,31200-00,Комиссия за выполнение функций агента валютног...,BANK_SERVICE
498,07.11.2024,18200-00,За тур.поездку по договору №75-04243Г от 24/04...,SERVICE


Проверим, есть ли в данных дублирующиеся строки.

In [70]:
print(f'Количество дубликатов в датасете: {data_markedup.duplicated().sum()}')

Количество дубликатов в датасете: 1


Как видим, в данных есть одна повторяющаяся строка, удалим ее.

In [71]:
data_markedup = data_markedup.drop_duplicates()

data_markedup.shape[0]

499

Посмотрим информацию о датасете.

In [72]:
data_markedup.info()

<class 'pandas.core.frame.DataFrame'>
Index: 499 entries, 1 to 500
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         499 non-null    object
 1   cash         499 non-null    object
 2   description  499 non-null    object
 3   type         499 non-null    object
dtypes: object(4)
memory usage: 19.5+ KB


В датасете нет пропусков, поэтому их не нужно предобрабатывать.

Взглянув на датасет, можно понять, что столбец "cash" с суммой денег имеет неунифицированный вид: значения могут быть записаны через запятую, через дифис, с разделителем тысяч в виде пробела.

Напишем функцию по унифицированию суммы.

In [ ]:
def unify_cash_format(cash_values: List[str]) -> List[float]:
    '''
    Функция принимает список строк, представляющих денежные суммы в различных форматах
    и преобразует их в унифицированный формат float. Функция обрабатывает различные разделители,
    такие как запятые, точки и дефисы, и гарантирует, что все значения правильно преобразованы
    в тип float.

    param cash_values (List[str]): Список строк, где каждая строка представляет собой денежную сумму.

    return unified_values (List[float]): Список float, где каждый float является унифицированным представлением денежной суммы.
    '''
    unified_values = []  # Список для хранения унифицированных значений
    for value in cash_values:
        # Проверяем, является ли значение строкой
        if isinstance(value, str):
            # Заменяем запятые на точки для корректного преобразования десятичных дробей
            value = value.replace(',', '.')
            # Удаляем дефисы
            value = value.replace('-', '.')
            # Удаляем разделители тысяч
            value = value.replace(' ', '.')
        try:
            unified_value = float(value)   # Преобразуем значение в float
            unified_values.append(unified_value)  # Добавляем преобразованное значение в список
        except ValueError:
            # Обрабатываем ошибки преобразования
            print(f"Предупреждение: Не удалось преобразовать значение '{value}' в float.")
            unified_values.append(0.0)  # Добавляем значение по умолчанию в случае ошибки
    return unified_values  # Возвращаем список унифицированных значений

Напишем функцию по полной предобработке датасета, которая:
- загружает файл с набором данных; 
- удаляет повторяющиеся строки;
- унифицирует колонку с суммой;
- удаляет колонку с датой, так как она имеет всего одно уникальное значение и не несет никакой информации (скорее всего, это значение является датой выгрузки данных из базы).

In [83]:
def preprocess_data(file_path: str, columns: List[str], unified_column: str, droped_column: str) -> pd.DataFrame:
    '''
    Эта функция загружает данные из TSV файла, обрабатывает столбец с денежными суммами,
    унифицируя их формат, и возвращает обновленный DataFrame.

    param file_path (str): Путь к TSV файлу, содержащему данные.
    param columns (List[str]): Список названий столбцов предобрабатываемого датафрейма. 

    return data (pd.DataFrame): Обновленный DataFrame с унифицированными денежными суммами в столбце 'cash'.
    '''
    
    # Проверка наличия unified_column и droped_column в columns
    if unified_column not in columns:
        raise ValueError(f"Столбец '{unified_column}' не найден в списке столбцов.")
    if droped_column not in columns:
        raise ValueError(f"Столбец '{droped_column}' не найден в списке столбцов.")
    
    # Загрузка данных из TSV файла
    data = pd.read_csv(file_path, encoding='utf-8', header=None, 
                                 names=columns, 
                                 sep='\t', index_col=0)

    # Удаление дубликтов
    data = data.drop_duplicates()
    data = data.reset_index(drop=True)

    # Применение функции к столбцу unified_column
    unified_cash = unify_cash_format(data[unified_column])

    # Обновление столбца unified_column в data
    data[unified_column] = unified_cash

    # Удаление столбца droped_column из data
    data = data.drop(droped_column, axis=1)

    return data  # Возвращаем обновленный DataFrame

Предобработаем размеченный датасет с помощью функции для предобработки и сохраним его.

In [81]:
data_markedup_path = '../data/payments_training.tsv'
save_data_markedup_clear_path = '../data/data_markedup_clear.tsv'

processed_data_markedup = preprocess_data(data_markedup_path, ['index', 'date', 'cash', 'description', 'type'], 'cash', 'date')

processed_data_markedup.to_csv(save_data_markedup_clear_path, sep='\t', index=False)
processed_data_markedup

,cash,description,type
0,15300.0,За участие в конференции в г. Майкоп по догово...,SERVICE
1,40200.0,За оказание услуг по договору №79-02726В от 01...,SERVICE
2,1440.0,Оплата за Порошок стиральный Ariel Color autom...,NON_FOOD_GOODS
3,240000000.0,Возврат денежных средств по договору займа №04...,LOAN
4,1360000.0,"Оплата Дог №452 от 13/03/2021, согл. Сч 0745-2...",NOT_CLASSIFIED
...,...,...,...
494,2610.0,Оплата налогов,TAX
495,31200.0,Комиссия за выполнение функций агента валютног...,BANK_SERVICE
496,18200.0,За тур.поездку по договору №75-04243Г от 24/04...,SERVICE
497,2870000.0,"Оплата по договору №095 от 24.02.2025г, счету ...",FOOD_GOODS


Аналогично предобработаем и сохраним неразмеченный датасет.

In [82]:
data_path = '../data/payments_main.tsv'
save_data_clear_path = '../data/data_clear.tsv'

processed_data = preprocess_data(data_path, ['index', 'date', 'cash', 'description'], 'cash', 'date')

processed_data.to_csv(save_data_clear_path, sep='\t', index=False)
processed_data

,cash,description
0,40500.0,За тур.поездку по договору №001 от 27.01.2023г
1,32600.0,За оказание услуг по договору №53Б-02746 от 23...
2,4710.0,Оплата штрафа
3,30900.0,Лечение по договору №Д-00359/24 от 08.03.2025
4,13200.0,Оплата основного долга за период с 16.12.2024г...
...,...,...
24753,3300000.0,Оплата по договору №Е01905 от 25.02.2023. сырь...
24754,2270.0,гос.услуга
24755,1750000.0,Предоставление кредита по договору №Д-00803/03...
24756,3560.0,Оплата гос. пошлины
